In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv("all_articles.csv")
df.head()

,PageNumber,Year,Title_TR,Title_EN,Abstract_TR,Abstract_EN,Keywords_TR,Keywords_EN
0,1,2020-2021,İnsansız Hava Araçları için Manyetik Rezonans ...,Magnetic Resonance Coupling Charging Station D...,İnsansız Hava Araçları (İHA) çeşitli alanlarda...,Unmanned aerial vehicles (UAVs) are electronic...,"insansız hava aracı (İHA), lityum batarya, man...","unmanned aerial vehicle (UAV), lithium battery..."
1,6,2020-2021,İş Modeli Kanvas ve Yakın İş Modellerinin Savu...,Evaluation of Business Model Canvas and Simila...,Savunma ve havacılık sanayinin rekabetçi koşul...,Competitive environment of aerospace and defen...,"savunma ve uçak sanayii, kanvas iş modeli, üni...","aerospace and defense industry, canvas busines..."
2,11,2020-2021,Hava Muharebesinde Otonom Savunma Algoritmasın...,Development of Autonomous Defense Algorithm in...,"Bu çalışma kapsamında, temel hava muharebesi m...","Within the scope of this study, an autonomous ...","bire-bir hava muharebesi, kural tabanlı yöntem...","one-to-one air combat, ruled-based method, bas..."
3,16,2020-2021,Modernization Projects,Uçak Modifikasyon-Modernizasyon Projelerinde A...,"Bu makalede, Türk Havacılık ve Uzay Sanayii LI...","In this article, there is information about th...","artırılmış gerçeklik, model tabanlı izleme, iş...","augmented reality, model-based tracking, marke..."
4,21,2020-2021,Döner Kanatlı Özgün Bir İHA Tasarımı ve Uçuş K...,Optimization of Indigenous Designed Flight Con...,Bu çalışmada 4 rotorlu döner kanatlı bir İnsan...,"In this study, a small 4-rotor X-wing UAV is d...","İHA, motor, uçuş kontrol kartı, özgün yazılım.","UAV, motor, flight controller, indigenous soft..."


In [3]:
df.isnull().sum()

PageNumber     0
Year           0
Title_TR       0
Title_EN       0
Abstract_TR    0
Abstract_EN    0
Keywords_TR    0
Keywords_EN    0
dtype: int64

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 666 entries, 0 to 665
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PageNumber   666 non-null    int64 
 1   Year         666 non-null    object
 2   Title_TR     666 non-null    object
 3   Title_EN     666 non-null    object
 4   Abstract_TR  666 non-null    object
 5   Abstract_EN  666 non-null    object
 6   Keywords_TR  666 non-null    object
 7   Keywords_EN  666 non-null    object
dtypes: int64(1), object(7)
memory usage: 41.8+ KB


In [5]:
# Gereksiz sütunları kaldır, şimdilik sadece makalelerin türkçe özelliklerini kullanacağız.
df = df.drop(columns=[
    "PageNumber",
    "Title_EN",
    "Abstract_EN",
    "Keywords_EN"
])

In [6]:
df.tail()

,Year,Title_TR,Abstract_TR,Keywords_TR
661,2023-2024,Yüksek Hassasiyetli LIDAR Tabanlı Erken Yangın...,"Bu çalışmada, günümüzde önemli ve hayati bir y...","Havacılık, LIDAR, yangın tespiti, IHA,algoritma"
662,2023-2024,Yüksek Hızlı Uçaklarda Aerodinamik Isınma,Bu çalışma kapsamında yüksek hızlı uçaklarda a...,Viskoz; Viskoz Olmayan; Sürtünme; Aerodinamik;...
663,2023-2024,Yüksek Hızlı Uçaklarda Meydana Gelen Aerodinam...,"Bu çalışmada, farklı irtifa ve Mach sayılarınd...","Aerodinamik Isınma, İrtifa, Mach Sayısı, Ansys..."
664,2023-2024,Yüksek İrtifada Görev Yapan Hava Araçlarında G...,"Bu çalışma, Küresel Konum Belirleme Sistemi (G...","INS, GNSS, VOR, DME, EKF, GPS- JAMMING"
665,2023-2024,Yüksek Performanslı EMI Koruması için Parafin,"Bu çalışmada, elektromanyetik girişim kalkanla...","Solvotermal, Genişletilmiş grafit, Demir oksit..."


In [7]:
# Metin Ön İşleme
# Önemli nokta: Transformer tabanlı dil modellerinin bağlamsal yapısı göz önünde
# bulundurularak ağır ön işleme adımlarından kaçınılmıştır.
# Bu nedenle yalnızca:
# - Küçük harfe çevirme
# - Gereksiz boşlukların temizlenmesi
# - Anlamsal katkısı olmayan özel karakterlerin (#, %, vb.) kaldırılması
# işlemleri uygulanmıştır.

In [8]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"\s+", " ", text)          # fazla boşluklar
    text = re.sub(r"[#%&*_=+<>]", "", text)   # anlamsız özel karakterler
    return text.strip()

In [9]:
df["title_tr_clean"] = df["Title_TR"].apply(clean_text)
df["abstract_tr_clean"] = df["Abstract_TR"].apply(clean_text)
df["keywords_tr_clean"] = df["Keywords_TR"].apply(clean_text)

In [10]:
# Başlık, özet ve anahtar kelimeler tek bir metin altında birleştirilerek
# projenin anlamsal bağlamını daha bütüncül şekilde temsil eden
# birleşik bir metin yapısı (combined_text) oluşturulmuştur.

df["combined_text"] = (
    df["title_tr_clean"] + " " +
    df["abstract_tr_clean"] + " " +
    df["keywords_tr_clean"]
)

In [11]:
df["combined_text"].iloc[4]

'döner kanatlı özgün bir i̇ha tasarımı ve uçuş kontrol kartının yerli yazılım ile optimizasyonu bu çalışmada 4 rotorlu döner kanatlı bir i̇nsansız hava aracı (i̇ha) tasarlanmıştır. gövdesi karbon fiberden kare şeklinde iki plakadan, kolları ise alüminyum kare profilden yapılmıştır. x-tipi bir i̇ha olan tasarımın ağırlığı toplamda görev mekanizması hariç 1300 gr, görev mekanizması dâhil 1700 gr ve 300 gr su ile görev sırasında maksimum 2000 gr olacak şekilde optimize edilmiştir. yaklaşık boyut motordan motora 50cm olup tasarıma ait uçuş kontrol kartı, özgün yazılımı ve yer kontrol yazılımı takım tarafından gerçekleştirilmiş, acil durumda elektrik aksamının kapatılması için önlemler alınmıştır. i̇ha, motor, uçuş kontrol kartı, özgün yazılım.'

In [12]:
# kaydetme
df.to_csv("articles_clean.csv", index=False)